In [25]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings(action='ignore')
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler

In [26]:
tierList = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'EMERALD', 'DIAMOND', 'MASTER', 'GRANDMASTER', 'CHALLENGER']
modelList = ['rf', 'lgbm', 'cat', 'et']
df = pd.read_csv(f'../Dataset/win_10/10_Grandmaster.csv')
# win__df = pd.read_csv(f'../Dataset/win_10/10_Grandmaster.csv')
# lose__df = pd.read_csv(f'../Dataset/lose_10/10_Grandmaster.csv')

In [27]:
global win_df, lose_df

df['FirstDragon_AIR_DRAGON'] = np.where((df['Diff_FirstDRAGON'] == 1) & (df['dragonType'] == 1), 1, 0)
df['FirstDragon_EARTH_DRAGON'] = np.where((df['Diff_FirstDRAGON'] == 1) & (df['dragonType'] == 2), 1, 0)
df['FirstDragon_FIRE_DRAGON'] = np.where((df['Diff_FirstDRAGON'] == 1) & (df['dragonType'] == 3), 1, 0)
df['FirstDragon_WATER_DRAGON'] = np.where((df['Diff_FirstDRAGON'] == 1) & (df['dragonType'] == 4), 1, 0)
df['FirstDragon_HEXTECH_DRAGON'] = np.where((df['Diff_FirstDRAGON'] == 1) & (df['dragonType'] == 5), 1, 0)
df['FirstDragon_CHEMTECH_DRAGON'] = np.where((df['Diff_FirstDRAGON'] == 1) & (df['dragonType'] == 6), 1, 0)

# 불필요한 기존 칼럼 삭제
df = df.drop(['dragonType'], axis=1)
# win__df = win__df.drop(['dragonType'], axis=1)
# lose__df = lose__df.drop(['dragonType'], axis=1)

win_df = df[['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-A', 'Diff_WARDplaced', 'Diff_WARDkill', 
           'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 
           'Diff_FirstBLOOD', 'FirstDragon_AIR_DRAGON', 'FirstDragon_EARTH_DRAGON', 
           'FirstDragon_FIRE_DRAGON', 'FirstDragon_WATER_DRAGON', 'FirstDragon_HEXTECH_DRAGON', 'FirstDragon_CHEMTECH_DRAGON']]
lose_df = df[['Diff_LV', 'Diff_CS', 'Diff_jglCS', 'Diff-K', 'Diff-A', 'Diff_WARDplaced', 'Diff_WARDkill', 
           'Diff_Inhibitor', 'Diff_TOWERkill', 'Diff_FirstDRAGON', 'Diff_FirstHERALD', 'Diff_Firsttower', 
           'Diff_FirstBLOOD', 'FirstDragon_AIR_DRAGON', 'FirstDragon_EARTH_DRAGON', 
           'FirstDragon_FIRE_DRAGON', 'FirstDragon_WATER_DRAGON', 'FirstDragon_HEXTECH_DRAGON', 'FirstDragon_CHEMTECH_DRAGON']]

display(win_df)
# display(lose_df)

,Diff_LV,Diff_CS,Diff_jglCS,Diff-K,Diff-A,Diff_WARDplaced,Diff_WARDkill,Diff_Inhibitor,Diff_TOWERkill,Diff_FirstDRAGON,Diff_FirstHERALD,Diff_Firsttower,Diff_FirstBLOOD,FirstDragon_AIR_DRAGON,FirstDragon_EARTH_DRAGON,FirstDragon_FIRE_DRAGON,FirstDragon_WATER_DRAGON,FirstDragon_HEXTECH_DRAGON,FirstDragon_CHEMTECH_DRAGON
0,0,29,6,5,3,-22,-1,0,0,1,-1,0,1,0,0,0,0,0,1
1,1,3,-8,2,4,-1,1,0,0,1,1,0,1,0,0,0,0,1,0
2,2,56,-7,5,3,3,3,0,0,0,1,0,1,0,0,0,0,0,0
3,-1,-23,-3,-6,-6,6,2,0,0,0,-1,0,-1,0,0,0,0,0,0
4,0,42,-16,5,6,26,-2,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6360,0,24,-10,0,2,3,4,0,0,0,-1,0,-1,0,0,0,0,0,0
6361,3,17,16,-3,-2,8,-1,0,0,0,1,0,-1,0,0,0,0,0,0
6362,-1,39,-9,-1,-3,-29,1,0,0,-1,1,0,1,0,0,0,0,0,0
6363,0,33,-14,5,3,-37,7,0,0,0,-1,0,-1,0,0,0,0,0,0


In [28]:
# def mergeCol(colName):
#     global win_df, lose_df
#     win_df[colName] = (win_df[f'{colName}_ad']+win_df[f'{colName}_top']+win_df[f'{colName}_jgl']+win_df[f'{colName}_mid']+win_df[f'{colName}_sup'])
#     lose_df[colName] = (lose_df[f'{colName}_ad']+lose_df[f'{colName}_top']+lose_df[f'{colName}_jgl']+lose_df[f'{colName}_mid']+lose_df[f'{colName}_sup'])
#     win_df= win_df.drop([f'{colName}_ad', f'{colName}_top', f'{colName}_jgl', f'{colName}_mid', f'{colName}_sup'],axis=1)
#     lose_df= lose_df.drop([f'{colName}_ad', f'{colName}_top', f'{colName}_jgl', f'{colName}_mid', f'{colName}_sup'],axis=1)
#     return win_df, lose_df

# win_df, lose_df = mergeCol('Kill')
# # win_df, lose_df = mergeCol('Death')
# win_df, lose_df = mergeCol('Asisst')
# win_df, lose_df = mergeCol('LV')
# win_df, lose_df = mergeCol('CS')
# win_df, lose_df = mergeCol('jglCS')
# # win_df, lose_df = mergeCol('GOLD')

# print(win_df['Diff-K'])
# print(lose_df['Diff-K'])
# lose_df['Diff-K'] = lose_df['Diff-K']*-1
# print(win_df['Diff-K'])
# print(lose_df['Diff-K'])
# win_df['Diff-K'] = win_df['Diff-K'] - lose_df['Diff-K'] 
# print(win_df['Diff-K'])
lose_df['Diff-K'] = win_df['Diff-K']*-1
# print(lose_df['Diff-K'])
# # win_df['Death'] = win_df['Death'] - lose_df['Death'] 
# # lose_df['Death'] = win_df['Death']*-1
# win_df['Diff_LV'] = win_df['Diff_LV']-lose_df['Diff_LV']
lose_df['Diff_LV'] = win_df['Diff_LV']*-1
# win_df['Diff_CS'] = win_df['Diff_CS'] - lose_df['Diff_CS'] 
lose_df['Diff_CS'] = win_df['Diff_CS']*-1
# win_df['Diff_jglCS'] = win_df['Diff_jglCS'] - lose_df['Diff_jglCS'] 
lose_df['Diff_jglCS'] = win_df['Diff_jglCS']*-1
# win_df['GOLD'] = win_df['GOLD'] - lose_df['GOLD'] 
# lose_df['GOLD'] = win_df['GOLD']*-1

# win_df.head()

print(win_df['Diff-K'])
print(lose_df['Diff-K'])

0       5
1       2
2       5
3      -6
4       5
       ..
6360    0
6361   -3
6362   -1
6363    5
6364   -6
Name: Diff-K, Length: 6365, dtype: int64
0      -5
1      -2
2      -5
3       6
4      -5
       ..
6360    0
6361    3
6362    1
6363   -5
6364    6
Name: Diff-K, Length: 6365, dtype: int64


In [29]:
# 이상치 제거
# def remove_outlier(input_data):
#     q1 = input_data.quantile(0.25) # 제 1사분위수
#     q3 = input_data.quantile(0.75) # 제 3사분위수
#     iqr = q3 - q1 # IQR(Interquartile range) 계산
#     minimum = q1 - (iqr * 1.5) # IQR 최솟값
#     maximum = q3 + (iqr * 1.5) # IQR 최댓값
#     # IQR 범위 내에 있는 데이터만 산출(IQR 범위 밖의 데이터는 이상치)
#     df_removed_outlier = input_data[(minimum < input_data) & (input_data < maximum)]
#     return df_removed_outlier

In [30]:
win_df.dropna(axis = 0, how = 'any', inplace = True)
lose_df.dropna(axis = 0, how = 'any', inplace = True)
win_df['result'] = 1
lose_df['result'] = 0
data = pd.concat([win_df, lose_df], axis=0)
data.result.value_counts(normalize=True)

print('Total data size =', data.shape[0])

Total data size = 12730


In [31]:
# 랜덤 포레스트
rf = RandomForestClassifier(max_features='sqrt', max_leaf_nodes=100)
# 라이트지비엠
lgbm = LGBMClassifier(n_estimators=100, max_depth=12, num_leaves=25, verbosity=0, min_child_samples=30)
# 캣부스트
cat = CatBoostClassifier(iterations=200, depth=5, learning_rate=0.2, l2_leaf_reg=50, verbose=0)
# 엑스트라트리
et = ExtraTreesClassifier(max_depth=7, max_features=None)

In [32]:
# print(f'{tier}의 {min}분 데이터')

modelList = [rf, lgbm, cat, et]
modelNameList = ["RandomForest", "LightGBM", "CatBoost", "ExtraTree"]
# fieldnameList = ["Model", "accuracy_score", "F1_score", "TN", "FP", "FN", "TP"]

x = data[data.columns.difference(['result'])]
y = data['result']

for i in range(4):
    model = modelList[i]

    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.2, random_state = 42)
    
    y_train = y_train.values.ravel()
    
    #0-1 정규화 진행
    mms = MinMaxScaler()
    #train 에만 fit적용, test나 예측할 새로운 데이터는 train으로 fit한것을 이용하여 변환
    x_train_scaled = mms.fit_transform(x_train)
    x_test_scaled = mms.transform(x_test)
    
    
    model.fit(x_train_scaled, y_train)
    pre_test = model.predict(x_test_scaled)
    pre_train = model.predict(x_train_scaled)
    if model == cat:
        printModel = f"CatBoost{model.get_params()}"
    else:
        printModel = model
    print(f"{modelNameList[i]} Train Accuracy", round(accuracy_score(y_train, pre_train)*100, 2), "%")
    print(f"{modelNameList[i]} Test Accuracy : ", round(accuracy_score(y_test, pre_test)*100, 2), "%")
    print(f"{modelNameList[i]} F1_score :", round(f1_score(y_test, pre_test)*100, 2), "%")
    print(f"{modelNameList[i]} ROC_AUC :", round(roc_auc_score(y_test, pre_test)*100, 2), "%")

RandomForest Train Accuracy 88.82 %
RandomForest Test Accuracy :  85.43 %
RandomForest F1_score : 85.52 %
RandomForest ROC_AUC : 85.44 %
LightGBM Train Accuracy 90.75 %
LightGBM Test Accuracy :  86.57 %
LightGBM F1_score : 86.7 %
LightGBM ROC_AUC : 86.58 %
CatBoost Train Accuracy 88.24 %
CatBoost Test Accuracy :  86.76 %
CatBoost F1_score : 86.88 %
CatBoost ROC_AUC : 86.77 %
ExtraTree Train Accuracy 86.09 %
ExtraTree Test Accuracy :  85.66 %
ExtraTree F1_score : 85.83 %
ExtraTree ROC_AUC : 85.67 %
